# Evaluation of a Convolutional Neural Network Architecture
Course project for the class _IoT Based Smart Systems_, carried out by _Riccardo Maria Pesce_ during Academic Year _2021-2022_, under the kind supervision of Professor _Maurizio Palesi_.

## Introduction

### Motivation
With the latest scientific and technological advancements that has taken place in the past few years, AI techniques have been employed in different fields, with great success. 

While Deep Learning Models are still trained on the cloud (using state-of-the-art computing machines with specialized hardware such as _GPU_ or _TPU_), it is becoming always more common to perform inference on the edge, i.e. on the devices itself, so as to reduce latency and optimize the usage of bandwith, a relevant issue for constrained devices.

### Objective
The objective of this thesis is to analyze a _CNN_ (Convolutional Neural Network) architecture performance on a constrained hardware, seeing how the mapping will affect the performance in terms of throughput and energy consumption.
In particular, these performance achievements are obtained in the following ways:

* Through reducing data movements, since communication is more expensive than computation in terms of energy nowadays. We can reduce data movements by either reducing the number of times memory is accessed, employing for instance DRAM which are nearby the _PEs_ (Processing Elements), or else we can compress data to a smaller number bits to represent it, thus making data movements cheaper. From these observations, we notice how __memory is the main bottleneck__.

* Maximizing PEs parallelism.

### Mapping
Mapping defines the order of execution of the MAC operations. The ordering can either be _temporal_ when operations are mapped serially on the same PE (i.e. the temporal order of execution), or _spatial_ when operations are mapped to multiple PE to execute in parallel.

### Introduction to Timeloop and Accelergy
In order to correctly design a DNN accelerator we need to cater for the different DNN architectures, and for each one of them we have to find an optimal mapping of these workloads onto specific hardware architectures.
This is what Timeloop and Accelergy do, and in particular:
* Timeloop generates a characterization of the energetical efficiency for each workload, through a mapper which finds the optimal way to plan operations on a specified architecture. To do so, Timeloop uses a coincise and unified representation of those core elements which are generally found in DNN accelerators.
* Accelergy, on the basis of the above created characterization, provides a pretty good estimate of energy consumption.

For a more complete introduction to Timeloop/Accelergy, please refer to [the official website](http://accelergy.mit.edu/tutorial.html).

## Simulation

### Objective

In this thesis, we want to find an efficient mapping of the _ResNet-50_ model onto different architectures.

### ResNet-18

We want to give a brief overview of the _ResNet-18_ architecture. 

![ResNet-18 Architecture](./assets/resnet18.png)

The main key points, as highlighted in the above picture, are:

* ResNet-18 architecture has __4 stages__.
* Input height and width must be multiple of 32 and channel width must be equal to 3.
* The main innovation of the ResNet architecture is the introduction of the _Identity Connection_ which allows the input feature map of some layer to skip some blocks and being summed to the output feature map of the skipped layers, before passing through the activation function (which is commonly the _ReLU_). This is a very succesful strategy to improve accuracy, thanks to the fact that we limit the _vanishing gradient_ problem which often occurs in very deep neural networks.
* ResNet uses Batch Normalization to mitigate the _Covariance Shift_ problem.

For a better understanding, please check out the original paper on [ArXiv](https://arxiv.org/abs/1512.03385).


### Accelerator architecture

We are going to try the ResNet-18 workload on different architectures. It is important to 



ModuleNotFoundError: No module named 'yaml'